# L13b: Drug Combination Design using Reinforcement Learning
In this lab, we apply Q-learning to design drug combinations that maximize therapeutic effectiveness while satisfying budget constraints. The goal is to learn an optimal policy that selects drug types and dosage levels through repeated interaction with an environment.

> __Learning Objectives:__
> 
> By the end of this lab, you will be able to:
> * **Implement Q-learning for constrained optimization**: Build a Q-learning agent that learns state-action value functions through temporal difference updates, demonstrating how reinforcement learning discovers optimal policies without explicit knowledge of environment dynamics.
> * **Model combinatorial drug design problems**: Encode drug cocktail design as a Markov decision process with discrete state spaces (dosage levels) and action spaces (drug selection), and apply Cobb-Douglas utility functions to evaluate therapeutic effectiveness under budget constraints.
> * **Extract policies from learned Q-values**: Derive optimal drug selection and dosage policies by analyzing converged Q-value tables, mapping each state (current dosage configuration) to the action (drug combination) that maximizes expected cumulative reward.

Let's get started!
___

## Problem
The Drug Cocktail Design Problem involves selecting a combination of drugs to create a cocktail that maximizes therapeutic effectiveness while staying within budget and safety constraints. Each drug has attributes such as dosage, mechanism of action, and side effects, which contribute to the overall effectiveness of the cocktail. The drug cocktail design problem can be formulated as the following optimization problem:
$$
\boxed{
\begin{align*}
\text{maximize} \quad & U(n_1, \dots, n_K) = \kappa(\gamma) \prod_{i=1}^{K} n_i^{\gamma_i} \\
\text{subject to} \quad & \sum_{i=1}^{K} c_i (n_i\;W) \leq B\quad\text{(budget constraint)}\\
& \sum_{i=1}^{K} n_i \leq S\quad\text{(safety constraint)}\\
& n_i^{\text{min}} \leq n_i \leq n_i^{\text{max}} \quad \forall i \in \{1, \dots, K\}
\end{align*}}
$$
where the objective function is a Cobb-Douglas utility function representing the effectiveness of the drug cocktail, and the constraints ensure that the cocktail stays within budget and safety limits. The design variables are the concentrations of each drug in the cocktail, denoted by $n_i$ for drug $i$ (units: mg/kg), $K$ is the total number of available drugs, and $W$ is the weight of the patient (units: kg). The $\kappa(\gamma)$ term is defined as:
$$
\begin{align*}
\kappa(\gamma) & = \begin{cases}
-1 & \text{if any } \gamma_i < 0 \\
1 & \text{if all } \gamma_i \geq 0
\end{cases}
\end{align*}
$$
where coefficients $\gamma_i$ represent the efficacy of each drug type, which must be learned from historical data or set based on expert knowledge.

The first set of constraints ensures that the total cost of the drug cocktail does not exceed a specified budget $B$, where $c_i$ is the cost per unit concentration (e.g., USD/mg) of drug $i$. The second set of constraints ensures that the total concentration of all drugs in the cocktail does not exceed a safety limit $S$. Finally, each drug concentration is bounded by its minimum and maximum allowable levels, denoted by $n_i^{\text{min}}$ and $n_i^{\text{max}}$, respectively.

In this lab, we'll implement the budget constraint only.
___

## Q-Learning Algorithm
Q-learning iteratively estimates the state action-value function $Q(s, a)$ by conducting repeated experiments $t=1,2,\ldots$ in the world $\mathcal{W}$. 
In each experiment, an agent in state $s\in\mathcal{S}$ takes action $a\in\mathcal{A}$, receives a reward $r$, and (potentially) transitions to a new state $s^{\prime}$. After each experiment $t$, the agent updates its estimate of $Q(s, a)$ using the update rule:
$$
\begin{equation*}
Q_{t+1}(s,a)\leftarrow{\underbrace{Q_{t}(s,a)}_{\text{old value}}}+\alpha_{t}\cdot\underbrace{\left(r+\gamma\cdot\max_{a^{\prime}\in\mathcal{A}}Q_{t}(s^{\prime},a^{\prime}) - Q_{t}(s,a)\right)}_{\text{new value}}\quad{t = 1,2,3,\ldots}
\end{equation*}
$$
where $0<\alpha_{t} <{1}$ is the learning rate parameter at time $t$, and $0<\gamma<{1}$ is the discount factor. 
We estimate the policy function $\pi:\mathcal{S}\rightarrow\mathcal{A}$ by selecting the action $a$ that maximizes $Q(s,a)$ at each state $s$:
$$
\begin{equation*}
\pi(s) = \arg\max_{a\in\mathcal{A}}Q(s,a)
\end{equation*}
$$

### Algorithm
Initialize $Q(s,a)$ arbitrarily for all $s\in\mathcal{S}$, and $a\in\mathcal{A}$.
Set the hyperparameters: learning rate $\alpha_{t}$, the discount factor $\gamma$, the exploration rate $\epsilon_{t}$, the maximum number of iterations $\texttt{maxiter}$, and the convergence tolerance $\delta$. Set $\texttt{converged}\gets\texttt{false}$. 

For $s\in\mathcal{S}$
1. Initialize the trial counter $t\gets{1}$
2. While $\texttt{converged} $ is $\texttt{false}$ __do__:
    1. Roll a random number $p\in[0,1]$. Compute $\epsilon_{t}={t^{-1/3}}\cdot\left(K\cdot\log(t)\right)^{1/3}$ where $K=|\mathcal{A}|$ is the number of actions.
    2. If $p\leq\epsilon_{t}$, choose a random (uniform) action $a_{t}\in\mathcal{A}$. Otherwise, choose a greedy action $a_{t} = \text{arg}\max_{a\in\mathcal{A}}{Q_{t}(s,a)}$.
    3. Take action $a_{t}$, observe the reward $r$ from the __world__ and transition to the next state $s^{\prime}$.
    4. Update the state-action-value function: $Q_{t+1}(s,a)\leftarrow{Q_{t}(s,a)}+\alpha_{t}\cdot\underbrace{\left(r+\gamma\cdot\overbrace{\max_{a^{\prime}\in\mathcal{A}}Q_{t}(s^{\prime},a^{\prime})}^{\text{one-step lookahead}} - Q_{t}(s,a)\right)}_{\text{new information}}$.
    5. Update the state $s\leftarrow{s^{\prime}}$, the learning rate $\alpha_{t+1}\leftarrow\alpha_{t}$, and the counter $t\leftarrow{t+1}$
    6. Convergence check: If $Q(s,a)$ has bounded change $\lVert{Q_{t+1}(s,a) - Q_{t}(s,a)}\rVert\leq\delta$, then the algorithm has converged. Set $\texttt{converged}\gets\texttt{true}$.
    7. Otherwise: if $t\geq\texttt{maxiter}$, then set $\texttt{converged}\gets\texttt{true}$ and notify the caller that the maximum iteration limit was reached without convergence. Proceed to next state.
    8. Otherwise: continue to the next iteration.
3. End While
4. End For

### Convergence
Q-learning converges to the optimal policy under two key theoretical conditions (assuming the Markov property holds for the environment):
* __Learning rate decay__: The learning rate $\alpha_{t}$ must satisfy $\sum_{t=0}^\infty \alpha_t(s, a) = \infty$ and $\sum_{t=0}^\infty \alpha_t^2(s, a) < \infty$ for all state-action pairs, ensuring sufficient initial updates while stabilizing over time. Setting $\alpha_{t+1} \gets \beta\alpha_{t}$ where $\beta<1$ is a common choice.
* __Infinite exploration__: All state-action pairs must be visited infinitely often. This condition holds for $\epsilon$-greedy policies with persistent exploration, i.e., $\epsilon_{t} > 0\,\,\forall{t}$.
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [5]:
include(joinpath(@__DIR__, "Include-student.jl")); # include the Include.jl file

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Implementation
Let's implement the `experiment(...)` function that is used by our reinforcement learning agent to evaluate drug cocktail designs. This function will compute the utility of a given drug cocktail based on the Cobb-Douglas utility function and apply the necessary constraints.

> __What is going on in the `experiment(...)` function?__
> 
> This function takes in a context object, which contains information about the drug types, their costs, and levels. It also takes in two integers, `s` and `a`, which represent the state and action, respectively. The function computes the utility of the drug cocktail based on the Cobb-Douglas utility function and checks if the cocktail meets the budget and safety constraints. If the constraints are met, it returns the computed utility; otherwise, it returns a penalty value.

Implement the `experiment(...)` function in the code cell below:

In [8]:
function experiment(context::MyExperimentalDrugCocktailContext, s::Int64, a::Int64)
    
    # TODO: Implement the experiment function to compute the utility of giving drug cocktail `a` with dose `s`
    throw(ErrorException("Oooops! The experiment function has not been implemented"))
end

experiment (generic function with 1 method)

### Constants
In this section, let's define some constants that will be used in our drug cocktail design problem. These constants include the number of drug types, their costs, and the levels of drug concentrations. See the comment next to each constant for its units, permitted values, and description.

In [10]:
K = 3; # number of drug types
m = 3;  # number of drug levels per type {high, nominal, low}
𝒜 = range(1, stop=2^K, step=1) |> collect; # action space: 2^K possible drug combinations (binary selection vector)
𝒮 = range(1, stop=m^K, step=1) |> collect; # state space: m^K possible dosage level configurations
W = 80.0; # weight of the patient in kg
B = 1000.0; # budget in USD

___

## Task 1: Setup the context model
In this task, we will set up the context model for our drug cocktail design problem. This involves defining a mutable struct that holds the necessary information about the drugs, their costs, and levels. 

> __Context model explanation:__ The context model stores the problem structure including the number of drug types `K`, the number of dosage levels `m` per drug, the drug effectiveness coefficients `γ` (which the agent does not know a priori), the unit costs per drug, the three dosage concentration levels (high, nominal, low) for each drug in mg/kg, the patient weight `W` in kg, and the total budget constraint `B` in USD. This encapsulates all the problem parameters that define the drug cocktail design environment.

We save the context information in the `contextmodel::MyExperimentalDrugCocktailContext` variable below:

In [13]:
contextmodel = let 

    # initialize - 
    costs = Dict{Int64, Float64}(); # cost per unit concentration (e.g., USD/mg) of drug i
    levels = Dict{Int64, NamedTuple}(); # levels of drug concentrations for drug i
    γ = Array{Float64}(undef, K); # effectiveness coefficients for each drug type

    # generate random cost data -
    for i ∈ 1:K
        costs[i] = rand(0.1:0.1:1.0); # random cost between 0.1 and 1.0 USD/mg
    end

    # generate random level data -
    for i ∈ 1:K
        high = rand(5.0:1.0:10.0);    # high concentration level in mg/kg
        nominal = rand(2.0:0.1:4.9);  # nominal concentration level in mg/kg
        low = rand(1.0:0.1:1.9);      # low concentration level in mg/kg
        levels[i] = (high=high, nominal=nominal, low=low);
    end

    # generate effectiveness coefficients -
    for i ∈ 1:K
        γ[i] = randn(); # random normal efficacy value (negative: inhibitory, positive: excitatory)
    end

    # build the model -
    model = build(MyExperimentalDrugCocktailContext, (

        K = K, # number of drug types
        m = m, # number of drug levels per type
        γ = γ, # effectiveness coefficients for each drug type (we don't know these a priori)
        B = B, # total budget in USD
        cost = costs, # cost per unit concentration (e.g., USD/mg) of drug i
        levels = levels, # levels of drug concentrations for drug i
        W = W # weight of the patient in kg
    ));
    
    model; # return the model
end;

Let's examine the randomly generated problem parameters for our drug cocktail design instance. This table displays the unit costs, dosage concentration levels, and effectiveness coefficients for each drug type:

In [15]:
let
    # initialize -
    df = DataFrame();
    
    # extract data from context model -
    for i ∈ 1:K
        row_df = (
            drug = i,
            unit_cost = contextmodel.cost[i],
            low_conc = contextmodel.levels[i].low,
            nominal_conc = contextmodel.levels[i].nominal,
            high_conc = contextmodel.levels[i].high,
            γ = contextmodel.γ[i]
        );
        push!(df, row_df);
    end
    
    # display the table -
    pretty_table(
        df;
        backend = :text,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 ------- ----------- ---------- -------------- ----------- ----------
   drug   unit_cost   low_conc   nominal_conc   high_conc          γ 
  Int64     Float64    Float64        Float64     Float64    Float64 
 ------- ----------- ---------- -------------- ----------- ----------
      1         0.8        1.2            2.4        10.0   0.151504
      2         0.1        1.0            2.4         9.0   0.158682
      3         0.5        1.2            4.0         9.0   -1.53923
 ------- ----------- ---------- -------------- ----------- ----------


___

## Task 2: Construct the Learning Agent Model
In this task, we will construct the learning agent model that evaluates different drug cocktail designs. This agent will run experiments using the `experiment(...)` function we implemented earlier and will learn to optimize the drug cocktail design over time to maximize utility while adhering to the constraints.

In [18]:
mylearningagent = let

    # initialize -
    γ = 0.95; # discount factor (for future rewards)
    α = 0.1;  # learning rate
    Q = Array{Float64}(undef, length(𝒮), length(𝒜)); # Q-value table

    # fill the Q-table with zeros -
    fill!(Q, 0.0); # fast way to fill an array with a value

    # build the learning agent model -
    model = build(MyQLearningAgentModel, (
        γ = γ, # discount factor
        α = α, # learning rate
        Q = Q, # Q-value table
        states = 𝒮, # state space
        actions = 𝒜 # action space
    ));

    model; # return the model
end

MyQLearningAgentModel([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  18, 19, 20, 21, 22, 23, 24, 25, 26, 27], [1, 2, 3, 4, 5, 6, 7, 8], 0.95, 0.1, [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

___

## Task 3: Let's let the agent learn!
In this task, we will allow the learning agent to interact with the environment and learn from its experiences. The agent will use the `experiment(...)` function to evaluate different drug cocktail designs and update its policy based on the observed rewards. 

> __What is going on in this code cell?__ We call the `solve(...)` method with our Q-learning agent, the context model, and the `experiment(...)` function as the world model. The agent will run up to 10,000 iterations, exploring different state-action pairs using epsilon-greedy selection. For each iteration, it takes an action (selecting drugs and dosages), observes the reward from the `experiment(...)` function (utility minus budget violation penalties), and updates the Q-value table using the temporal difference learning rule. The algorithm terminates when the maximum change in Q-values falls below the convergence tolerance δ = 0.0001 or when the maximum number of iterations is reached.

The `result` variable stores the trained Q-learning model containing the converged Q-value table `result.Q`, which encodes the expected cumulative reward for each state-action pair after learning.

In [21]:
result = solve(mylearningagent, contextmodel; 
    maxsteps = 10000, δ = 0.0001, worldmodel = experiment);

LoadError: Oooops! The experiment function has not been implemented

In [22]:
result.Q

LoadError: UndefVarError: `result` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

Let's extract the policy from our trained agent.

In [24]:
π(s) = mypolicy(result.Q)[s]; # define a policy function that returns the best action for state s

In [25]:
π(1)

LoadError: UndefVarError: `result` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

What's in the policy? Let's pick the best action for each state according to the learned Q-values. We'll display these in a table.

> __What's going on in this table?__ For a given state `s` (representing a specific dosage level configuration), we extract the optimal action `a` from the learned policy. We then decode the action into a binary vector to identify which drugs are selected, and decode the state to determine the dosage level (high, nominal, or low) for each selected drug. 
> 
> The table displays the drug index, dosage level, unit cost, concentration levels available, total spending for that drug (concentration × patient weight × unit cost), the effectiveness coefficient γ, and the Q-value for this state-action pair. This shows the recommended drug cocktail composition that maximizes expected utility while respecting budget constraints for each state (dose combinations).

The table reveals which drugs the agent recommends including in the cocktail, at which dosage levels, and the expected value of following this recommendation from the given state.

In [27]:
let

    # initialize -
    df = DataFrame();
    s = 10; # what state do we want to look at?
    a = π(s); # this gives me the best action *index* for state s
    N = 2^K; # number of actions
    M = m^K; # number of states
    γ = contextmodel.γ; # effectiveness coefficients for each drug type

    aᵢ = digits(a, base=2, pad=K); # action vector representation
    if (a == N)
        aᵢ = digits(a-1, base=2, pad=K); # adjust for all-drug case
    end
    S₊ = findall(x -> x == 1, aᵢ); # indices of drugs being administered

    for i ∈ eachindex(S₊)
        drug_index = S₊[i];
        level_index = digits(s, base=m, pad=K)[drug_index];

        level_str = "";
        if (level_index == 0)
            level_str = "high";
        elseif (level_index == 1)
            level_str = "nominal";
        elseif (level_index == 2)
            level_str = "low";
        end
        n = contextmodel.levels[drug_index];
        spend = 0.0;
        if level_index == 0
            spend = (W*n.high)*contextmodel.cost[drug_index];
        elseif level_index == 1
            spend = (W*n.nominal)*contextmodel.cost[drug_index];
        elseif level_index == 2
            spend = (W*n.low)*contextmodel.cost[drug_index];
        end

        push!(df, (
            state = s,
            action = a,
            drug = drug_index,
            level = level_str,
            unitcost = contextmodel.cost[drug_index],
            concentration = n,
            spend = spend,
            γ = γ[drug_index],
            q_value = result.Q[s, a]
        ));
    end

    # make a pretty table -
    pretty_table(
        df;
        backend = :text,
        fit_table_in_display_horizontally = false,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );

end

LoadError: UndefVarError: `result` not defined in `Main`
Suggestion: add an appropriate import or assignment. This global was declared but not assigned.

___

## Summary
In this lab, we applied Q-learning to solve the drug cocktail design problem, learning optimal policies for selecting drug combinations and dosage levels under budget constraints.

> __Key Takeaways:__
> 
> * **Q-learning for combinatorial optimization**: We implemented a Q-learning agent that iteratively estimates state-action value functions through temporal difference updates, converging to an optimal policy without requiring explicit knowledge of transition probabilities or reward functions. The algorithm balanced exploration (trying new drug combinations) with exploitation (selecting known effective combinations) using an epsilon-greedy strategy with decaying exploration rate.
> * **MDP formulation for drug design**: We encoded the drug cocktail problem as a Markov decision process with states representing dosage level configurations (high, nominal, low for each drug type), actions representing binary drug selection vectors, and rewards computed from Cobb-Douglas utility functions penalized for budget violations. This formulation captured both therapeutic effectiveness (through preference parameters γᵢ) and resource constraints (through budget limits).
> * **Policy extraction and interpretation**: We derived the optimal policy by selecting argmax actions from the converged Q-value table, mapping each state to the drug combination that maximizes expected cumulative reward. The learned policy encoded both drug selection decisions (which drugs to include) and dosage optimization (which concentration levels to use), demonstrating how reinforcement learning discovers structured solutions to constrained combinatorial problems.

Q-learning provides a framework for sequential decision-making in optimization problems where the relationship between actions and outcomes must be learned through experience rather than analytical derivation.
___